In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import mean_squared_error
from math import sqrt
import matplotlib.pyplot as plt
from matplotlib import style
from scipy.stats import norm
# allows currency formatting
import locale
locale.setlocale(locale.LC_ALL, 'en_CA.UTF-8')
%matplotlib inline
style.use('ggplot')

In [ ]:
#Ask for user name 
#Ask for user age
#Ask for how long of investment
# Ask for conservative: 30%, moderate: 50%, risky 80%
# Give total amount stocks investing in
# Build your portfolio
# standard 10 stocks will increase as upgraded
# conservative 3 growth, 7 value - out of Top 20 for both growth and value
# moderate 5 growth, 5 value - out of Top 20 for both growth and value
# risky 8 growth, 2 value - out of Top 20 for both growth and value
# save the similar stocks in a dataframe
# give their amount they would spend each year 
# give possible outcomes

In [81]:
cluster_rec = pd.read_csv('clusters_rec.csv')
growth_stocks = pd.read_csv('growth_stocks.csv')
value_stocks = pd.read_csv('value_stocks.csv')
predictions = pd.read_csv('prediction_gv.csv')
growth_stocks = growth_stocks.rename({'0':'symbol'}, axis = 1)
value_stocks = value_stocks.rename({'0':'symbol'}, axis = 1)


In [69]:
# cluster_ticker = pd.DataFrame(cluster_rec['symbol'])
# cluster_ticker.to_csv('/Users/flatironschool/UrPortfolio/Classification_Models/clusters_ticker.csv', index = False)

###  User Input - What is stock is Growth or Value 

In [83]:
stockz = input('Input a Stock to know if it is Growth or Value: ').upper()
test = predictions[predictions['symbol'] == stockz]
growth = test['Growth'].sum()
value = test['Value'].sum()
if growth > value:
    print(f'Through our fundamental anaylsis: {stockz} is a Growth Stock')
else:
    print(f'Through our fundamental anaylsis: {stockz} is a Value Stock')

Input a Stock to know if it is Growth or Value: a
Through our fundamental anaylsis: A is a Value Stock


In [160]:
if type('j') == int:
    print('true')

###  User Input - Time Horizon, Risk Tolerance

In [169]:
time_investing = input('How long would you like to invest for: ')

while True:
    try:
        time_investing = int(time_investing)
        if type(time_investing) == int:
            break
    except ValueError:
        print('Please use digits')
        time_investing  = input('How long would you like to invest for: ')


risk_tolerance = input(" ----------\n 1- Conservative\n 2- Moderate\n 3- Risky \n Please choose from the following: ").lower()
while True:
    if risk_tolerance == 'conservative' or risk_tolerance == '1':
        risk_tolerance = 30
        break
    elif risk_tolerance == 'moderate' or risk_tolerance == '2': 
        risk_tolerance = 50 
        break
    elif risk_tolerance == 'risky' or risk_tolerance == '3':   
        risk_tolerance = 80
        break
    else:
        print("invalid input try again\n ------")
        risk_tolerance = input(" 1- Conservative\n 2- Moderate\n 3- Risky \n Please choose from the following: ").lower()
    
                
risks = float(risk_tolerance)/100
print(risks)

How long would you like to invest for: 4
 ----------
 1- Conservative
 2- Moderate
 3- Risky 
 Please choose from the following: 4
invalid input try again
 ------
 1- Conservative
 2- Moderate
 3- Risky 
 Please choose from the following: 3
0.8


In [143]:
# risks = input('input your risk: ')
cluster = cluster_rec[cluster_rec['yr_variance'] <= risks]
cluster = cluster[cluster['sharpe_ratio'] >= 0]
cluster.sort_values('sharpe_ratio', ascending = True)
stocking = int(cluster.iloc[0][0])
#testing trial
rec_comp_main = cluster.drop(columns ='Unnamed: 0')
rec_test = rec_comp_main.drop(columns = ['symbol', 'Price','company','sector'])
pip = rec_test.sort_values('avg_yr_returns', ascending=False)
pip = pip.index[0]

In [6]:
scaler = StandardScaler()
scaler.fit(rec_test)
model_knn = NearestNeighbors(metric = 'cosine', algorithm= 'brute')
model_knn.fit(rec_test)

# KNN recommender system
test = [stocking]
dftest = rec_test.iloc[test,:]
distances, indices = model_knn.kneighbors(rec_test, n_neighbors=30)
stock = pd.DataFrame(indices.T)[test]
stock = stock.set_index(test)
stock = stock.merge(rec_comp_main, left_index = True, right_index = True)
testing_2 = stock[stock['Price'] <= 100].head(5)
testing_2

StandardScaler(copy=True, with_mean=True, with_std=True)

In [41]:
inpt_idx = pip
inpt = rec_test.loc[inpt_idx]
scores = pd.Series(index=rec_test.index)
scores.loc[inpt_idx] = -1
for idx, stock in rec_test.drop(index=inpt_idx).iterrows():
    diff = 0
    for feature in stock.index:
        diff += (inpt.loc[feature]-stock.loc[feature])**2
    scores.loc[idx] = diff
scores.sort_values()
scores = pd.DataFrame(scores.sort_values())
recommend_err = scores.merge(rec_comp_main, left_index = True, right_index = True)
recommend_err = recommend_err.rename({0: 'RMSE'}, axis = 1)
testing = recommend_err.head(10)

In [44]:
testing

,RMSE,symbol,avg_yr_returns,yr_variance,sharpe_ratio,cluster,Price,company,sector
521,-1.000000,UTX,0.280900,0.294438,0.186084,10,17.510000,Textron Inc.,Industrial Goods
427,0.000086,PXD,0.277993,0.287377,0.180847,10,6.319800,"Prologis, Inc.",Financial
202,0.005361,FB,0.233304,0.278072,0.132907,10,55.689999,"Facebook, Inc.",Technology
156,0.007157,DFS,0.236406,0.254064,0.126523,10,161.160004,Discover Financial Services,Financial
458,0.007475,SEE,0.223475,0.278855,0.123361,10,154.705002,ResMed Inc.,Healthcare
44,0.008005,AME,0.220707,0.280838,0.121299,10,28.523600,"Advanced Micro Devices, Inc.",Technology
71,0.008818,AYI,0.218289,0.278314,0.117980,10,1057.770020,"Acuity Brands, Inc.",Technology
563,0.010774,YUM,0.242299,0.225674,0.118592,10,118.125000,"Waste Management, Inc.",Industrial Goods
79,1.040107,BC,0.157207,0.237019,0.039421,11,245.544998,Brunswick Corporation,Consumer Goods
225,1.040410,GCI,0.151695,0.248162,0.039199,11,178.335007,"Gannett Co., Inc.",Services


## Testing Part 2

In [43]:
# Show user price paid per year
# Show user expected return
# Show user risk factor
price_per = float(testing['Price'].sum()) + 6
expected_return = float(testing['avg_yr_returns'].mean())
volatility = float(testing['yr_variance'].mean())
print(f'price per year : {price_per}')
print(f'expected return : {expected_return}')
print(f'risk : {volatility}')

price per year : 2029.683428287506
expected return : 0.2242275543176227
risk : 0.2662812751220686


In [34]:
pv = float(testing['Price'].sum()) + 6
expected_return = float(testing['avg_yr_returns'].mean())
volatility = float(testing['yr_variance'].mean())
time_horizon = int(time_horizon)
annual_addition = pv

print("\tReturn", "\t\tEnding Value".rjust(18))
for year in range(time_horizon):
    market_return = np.random.normal(expected_return, volatility)
    fv = pv * (1 + market_return) + annual_addition
    print("\t{}".ljust(10).format(round(market_return,4)), "\t{}".rjust(10).format(locale.currency(fv, grouping=True)))
    pv = fv

NameError: name 'testing' is not defined

In [384]:
# 4. Simulate portfolio ending market values 

sim = pd.DataFrame()
iterations = 5000


for x in range(iterations):
    pv = float(testing['Price'].sum()) + 6
    expected_return = float(testing['avg_yr_returns'].mean())
    volatility = float(testing['yr_variance'].mean())
    time_horizon = int(time_horizon)
    annual_investment = pv
    stream = []
    for i in range(time_horizon):
        end = round(pv * (1 + np.random.normal(expected_return,volatility)) + annual_investment,2)
        
        stream.append(end)

        pv = end

    
    sim[x] = stream

In [385]:
# 5. Sample first five streams of annual ending values
first_five = list(range(5))
sim[first_five]

,0,1,2,3,4
0,124.78,135.88,156.56,128.71,162.10
1,215.42,221.35,274.45,243.75,293.92
2,229.25,167.04,397.99,222.92,328.44
3,493.17,255.50,366.59,320.70,363.40
4,927.33,347.64,150.63,337.67,320.23


In [386]:
import plotly.graph_objects as go

# Create random data with numpy
import numpy as np


random_x = sim.index
random_y0 = sim[1]
random_y1 = sim[2]
random_y2 = sim[3]
random_y3 = sim[4]
random_y4 = sim[5]

# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(x=random_x, y=random_y0,
                    mode='lines+markers',
                    name='stream_1'))
fig.add_trace(go.Scatter(x=random_x, y=random_y1,
                    mode='lines+markers',
                    name='stream_2'))
fig.add_trace(go.Scatter(x=random_x, y=random_y2,
                    mode='lines+markers', 
                    name='stream_3'))

fig.add_trace(go.Scatter(x=random_x, y=random_y3,
                    mode='lines+markers',
                    name='stream_4'))
fig.add_trace(go.Scatter(x=random_x, y=random_y4,
                    mode='lines+markers', 
                    name='stream_5'))

fig.show()

In [387]:
# 7. Generate summary statistics with numpy functions
amount = len(sim)-1
print("Count:", len(sim.loc[amount]))
print("Mean: ", locale.currency(np.mean(sim.loc[amount]),grouping=True))
print("SD: ",locale.currency(np.std(sim.loc[amount]),grouping=True))
print("Max: ",locale.currency(np.max(sim.loc[amount]), grouping=True))
print("Min: ", locale.currency(np.min(sim.loc[amount]), grouping=True))

Count: 5000
Mean:  $589.56
SD:  $303.76
Max:  $2,486.44
Min:  -$81.11


In [388]:
#You can get a more comprehensive table of percentiles easily using numpy's percentile function
ending_values = sim.loc[amount]
p_tiles = np.percentile(ending_values,[5,10,15,25,50,75])
for p in range(len(p_tiles)):
    l = [5,10,15,25,50,75]
    print("{}%-ile:  ".format(l[p]).rjust(15),"{}".format(locale.currency(p_tiles[p], grouping=True)))
#     print(f"{l[p]}%-ile:  ",f"{locale.currency(p_tiles[p], grouping=True)}")

      5%-ile:   $213.08
     10%-ile:   $268.06
     15%-ile:   $308.33
     25%-ile:   $371.71
     50%-ile:   $527.24
     75%-ile:   $744.96
